# 2.8 通过微调提升模型的准确度与效率

## 🚄 前言

前面的课程中，介绍了如何搭建一个答疑机器人，并尝试通过优化提示词和RAG过程、扩展插件等方式提升其能力。不过你可能会发现，随着流程变得复杂，应用处理节点（依赖大模型）的增多，导致响应也在变慢。

在实际生产落地过程中，很多企业会尝试将一些任务单一的节点（如下图中执行NL2SQL的大模型），替换成参数量更小的模型，并对其进行微调，让它在特定的领域的表现可以对标更大参数规模的模型，来换取更快的响应速度，与更低的成本开销。

NL2SQL任务样例：

`模型输入：数据库的各种信息（各表信息、列信息） +  自然语言问题：有多少部门的负责人年龄超过56岁？`

`期望模型输出："SELECT count(*) FROM head WHERE age > 56"`

## 🍁 课程目标

学完本课程后，你将能够：

*   了解什么是模型微调
    
*   了解构建微调数据集的技巧
    
*   简单了解模型评测指标
    
*   通过案例，一步步体验如何微调

## 📖 课程目录
- [1. 模型小型化](#1------)
  * [1.1 模型小型化方案](#11--------)
    + [1.1.1 模型量化](#111-----)
    + [1.1.2 模型剪枝](#112-----)
    + [1.1.3 使用更小的模型](#113--------)
  * [🤔 1.2 从全参微调到高效微调](#---12-----------)
- [🛠️ 2. LoRA微调实践](#----2-lora----)
  * [2.1 环境准备](#21-----)
  * [2.2 微调数据准备](#22-------)
  * [2.3 微调任务目标](#23-------)
    + [2.3.1 尝试本地模型](#231-------)
    + [2.3.2 评测指标](#232-----)
    + [2.3.3 评测本地模型](#233-------)
  * [2.4 微调模型](#24-----)
  * [2.5 评测微调后的模型](#25---------)
    + [2.5.1 尝试微调后的模型](#251---------)
    + [2.5.2 评测微调后的模型](#252---------)
  * [2.6 参数矩阵融合](#26-------)
- [✅ 本节小结](#------)
- [扩展阅读](#----)
  * [开展微调任务前必须要关注的](#-------------)
  * [微调数据集构建策略](#---------)
- [🔥 课后小测验](#--------)

## 1. 模型小型化

在开始实验前，我们先设置好环境变量。

In [ ]:
# 加载百炼的 API Key 用于调用通义千问大模型
import os
from config.load_key import load_key
load_key()
print(f'''你配置的 API Key 是：{os.environ["DASHSCOPE_API_KEY"][:5]+"*"*5}''')

在2.5节构造了一个查询员工信息的工具，需要进行NL2SQL（将问题转化为查询数据库的SQL语句），这里来横向对比一下`qwen-plus`和`qwen2.5-1.5b-instruct`的SQL查询效果。

In [1]:
import os, time
import prettytable as pt
from openai import OpenAI


qwen_plus = "qwen-plus"
qwen_2_5_1_5b = "qwen2.5-1.5b-instruct"


# 定义一个员工查询函数
def query_employee_info(query, llm):
    client = OpenAI(
        api_key=os.getenv("DASHSCOPE_API_KEY"),
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    )

    completion = client.chat.completions.create(
        model=llm,
        messages=[
            {'role': 'system', 'content': '''你是一个擅长写SQL的人，你有一个表叫employees，记录公司的员工信息，这个表有department（部门）、name（姓名）、HR三个字段。
             department中有教育部门、后勤部门、技术部门。
             你需要根据用户输入生成sql语句进行查询,你一定不能生成sql语句之外的内容，也不要把```sql```这个信息加上。'''},
            {'role': 'user', 'content': query}])
    SQL_output = completion.choices[0].message.content
    if SQL_output == "SELECT COUNT(*) FROM employees WHERE department = '教育部门'":
        return SQL_output, "查询成功：教育部门共有66名员工。"
    if SQL_output == "SELECT HR FROM employees WHERE name = '张三'":
        return SQL_output, "查询成功：张三的HR是李四。"
    if SQL_output == "SELECT department FROM employees WHERE name = '王五'":
        return SQL_output, "查询成功：王五的部门是后勤部。"
    else:
        return SQL_output, "查询失败：抱歉，我暂时无法回答您的问题。"

# 初始化展示表格
table = pt.PrettyTable(["", "qwen-plus", "qwen2.5-1.5b"])
sql_row, ans_row, time_row, empty_row = ["SQL语句"], ["查询结果"], ["回答耗时"], ['','', '']

# 设置表格“qwen_plus” 一列
current_time = time.time()
ans = query_employee_info("教育部门有几个人", qwen_plus)
sql_row.append(ans[0])
ans_row.append(ans[1])
time_row.append(str(round(time.time() - current_time, 3)) + '秒')

# 设置表格“qwen2.5-1.5b” 一列
current_time = time.time()
ans = query_employee_info("教育部门有几个人", qwen_2_5_1_5b)
sql_row.append(ans[0])
ans_row.append(ans[1])
time_row.append(str(round(time.time() - current_time, 3)) + '秒')

table.add_rows((ans_row, empty_row, time_row, empty_row, sql_row))
table.set_style(pt.DOUBLE_BORDER)
print(table)

╔══════════╦══════════════════════════════════════════════════════════════╦══════════════════════════════════════════╗
║          ║                          qwen-plus                           ║               qwen2.5-1.5b               ║
╠══════════╬══════════════════════════════════════════════════════════════╬══════════════════════════════════════════╣
║ 查询结果 ║               查询成功：教育部门共有66名员工。               ║ 查询失败：抱歉，我暂时无法回答您的问题。 ║
║          ║                                                              ║                                          ║
║ 回答耗时 ║                           0.997秒                            ║                 0.741秒                  ║
║          ║                                                              ║                                          ║
║ SQL语句  ║ SELECT COUNT(*) FROM employees WHERE department = '教育部门' ║                  ```sql                  ║
║          ║                                                              ║  SELECT COUNT(*) AS num

可以看到，模型的参数减少到1.5B(billion)后，NL2SQL任务的答案虽然很接近了，但仍然有调教改进的空间。同时模型参数变小后也有多个显著优点：占用内存降低、使用成本降低、响应时间变快。总而言之就是能提高SLA，还能省钱。（如果正确率能一致）

通俗地讲，一个通用的大模型很全能，就像旗舰手机一样，在摄像、处理器等几乎所有领域都很优秀，适合所有人但价格昂贵。而大部分用户的预算并不能负担旗舰手机，往往会选择能满足自己需求并且价格更加便宜的手机。

这种花钱花在刀刃上的场景在实际应用中非常常见，所以能将模型小型化的解决方案是非常有必要的，那让我们来看一下有哪些可行方案。

### 1.1 模型小型化方案

由于`qwen-plus`的模型参数规模未公开，暂且假设它有70B的参数规模。

**任务目标：将模型参数缩减至1B左右的规模，即原来的1/70（1.5%）。**



#### 1.1.1 模型量化

模型量化是一种最简单，也是如今很常用的模型压缩技术，核心就是将网络中的所有参与计算的值都降低精度，近似表示结果。图中这里展示了如何将FP32（32bit）的数使用INT4（4 bit）进行近似。

<img src="https://alidocs.oss-cn-zhangjiakou.aliyuncs.com/res/yBRq1MA2ppZKqdv1/img/de9d3691-2dd9-49c7-82df-36525eea1ea3.png" width="300">

模型量化后相当于把模型规模减少至1/8(12.5%)，虽然这已经是很了不起的压缩了，但并不能满足需求。不过有趣的是虽然只使用了12.5%的资源，但对模型的精度影响非常小（ <2%）。



#### 1.1.2 模型剪枝

这里我们可以很形象地把模型比作一棵树，我们有两种修剪枝叶、减少模型参数的方法：

参数剪枝，像是从树上摘掉一些不重要的树叶，虽然对每根树枝都影响不大，但减掉的树叶总体会很多。

结构剪枝，像是修剪掉不重要的整根树枝，而不是单独的树叶。

<img src="https://alidocs.oss-cn-zhangjiakou.aliyuncs.com/res/yBRq1MA2ppZKqdv1/img/bc6675a9-cc6b-4fb9-886c-6f7ca20c6019.png" width="500">


不过非常遗憾的是模型剪枝在基本不影响模型性能的前提下，最多只能减少一半左右（50%）的参数。所以模型剪枝也无法满足我们的要求。



#### 1.1.3 使用更小的模型

综合来看，精简原有模型（qwen-plus）很难达到预期的目标（参数减少至1.5%）。

也许我们应该换个思路，是否可以直接使用参数规模符合要求的小参数模型？比如使用`Qwen2.5-0.5B`或者`Qwen2.5-1.5B`来替代原来的`qwen-plus`。但使用小参数模型相对于大参数模型在NL2SQL任务上能力的不足该如何弥补呢？

**我们可以通过模型微调来定向提升模型在NL2SQL任务的能力。**模型微调可以在保持模型相对轻量化的同时，显著提升在某些特定任务上的准确度和效率。

微调的准备一般分为以下几步：

1.  **选择被微调模型**：本节中将以`qwen2.5-1.5b-instruct`为例作为被微调的模型。
    
2.  **准备微调数据集**：本节已经准备好了数据集。（关于数据集构建更多的介绍请前往拓展阅读）
    
3.  **选择微调方法**：根据任务需求与可用硬件配置选择不同的模型微调方法，但大致分为全参微调和高效微调两种方式，接下来我们将进一步探究微调方法的选择。

### 🤔 1.2 从全参微调到高效微调

在开始微调前，先粗略计算一下微调`qwen2.5-1.5b-instruct`时需要的显存资源。

$模型参数内存=1,500,000,000×4 字节=6,000,000,000 字节=5.73 GB$（一个浮点数4字节）

一般对模型进行全参微调时，大概需要模型参数内存的7~8倍，也就是**46GB**的显存。这个显存占用基本上超过了大多数显卡的配置，也超过了我们后面要用到的GPU实验环境显存。

想象一下如果模型规模是30B、60B，对显存的要求将非常高。那有没有什么办法可以节省显存使用呢？

当然有，业界研究出了很多高效微调 PEFT（Parameter-Efficient Fine-Tuning）技术。PEFT的核心思想就是在少训练甚至不训练被微调模型参数的情况下，达到与全参微调相似的效果。

在PEFT的发展过程中，代表方法有：

1.   Freeze，该方法是最早的PEFT方法。它在微调时冻结模型的大部分参数，仅训练模型中的小部分参数（比如最后几层神经网络），来快速适应特定任务的需求。但实践证明Freeze的微调效果欠佳。
    
2.  Adapter Tuning，该方法会在原有的模型架构上，在某些位置之间插入Adapter层，微调时模型原有参数不会被训练，只训练这些Adapter层，而原先的参数不会参与训练。  
    该方法可以在只额外增加3.6%参数的情况下达到与全参数微调相似的效果。但它的实现比较复杂，需要针对被微调模型的结构进行定制，无法标准化，“手工感”比较强。

    <img src="https://alidocs.oss-cn-zhangjiakou.aliyuncs.com/res/yBRq1MA2ppZKqdv1/img/f22b010b-6d6c-447c-8cb6-83b821f011bd.png" width="600">

    
3.  LoRA（Low-Rank Adaptation）是如今最普遍使用的PEFT方法。该方法通过引入一组低秩矩阵来调整被微调模型的权重，从而不改变模型结构，也不新增神经网络层。  
    微调时只额外引入的低秩矩阵`A * B`，使得微调的参数量下降`2 * r / d`倍。一般的`d = 1024、r = 16`，这样微调参数量是被训练模型的3.1%。而微调效果经常能超过全参微调。  
    <table><tr><td bgcolor=white>
    <img src="https://alidocs.oss-cn-zhangjiakou.aliyuncs.com/res/oJGq765B3YDrnAKe/img/b2e2caf3-e8da-4ac8-b57d-abbb05f932c4.png" width="800"></tr></table>

    如果你对LoRA的更多细节感兴趣，可以阅读：[_LoRA: Low-Rank Adaptation of Large Language Models_](https://arxiv.org/abs/2106.09685)
    

现在最常用的微调方法就是LoRA及其相关衍生技术，因为它相对于其他高效微调方法有非常多的优点：

1.  LoRA直接作用于模型最底层的矩阵运算，通用性强，不需要像Adapter Tuning一样必须了解被微调模型的结构。
    
2.  微调的成果只存储在"微调参数矩阵"中，不会破坏被微调模型的结构也不会改变被微调模型参数。
    
3.  参数集成与分离实现方便，便于开发者快速实现和集成。
    

本文也将以LoRA微调来开展微调实验。

## 🛠️ 2. LoRA微调实践

实践目标：使用准备好的模型与NL2SQL微调数据集进行微调实践，观察微调前后，模型在NL2SQL能力的提升。

NL2SQL任务样例：（数据库的信息对模型输出是必要的，它包括表名、列名、主键、外键等信息，这些信息帮助模型正确地组织SQL语句）

`模型输入：数据库的各种信息（各表信息、列信息） +  自然语言问题：有多少部门的负责人年龄超过56岁？`

`期望模型输出："SELECT count(*) FROM head WHERE age > 56"`

本课程使用[ms-swift](https://github.com/modelscope/ms-swift/tree/main)（Modelscope Scalable lightWeight Infrastructure for Fine-Tuning）框架，它是魔搭社区专门为模型训练开发的开源框架，该框架支持350+ LLM和90+ MLLM（多模态大模型）的训练(预训练、微调、对齐)、推理、评测和部署。



### 2.1 环境准备

由于模型微调对硬件性能要求较高，因此你需要使用PAI的交互式建模（DSW）重新创建一个包含GPU的实例，让你可以更快地完成微调任务。

请在另一个窗口打开课程章节“1\_0\_计算环境准备”的内容作为参考，环境准备操作可以参考[计算环境准备](https://edu.aliyun.com/course/3130200/lesson/343310285)中的**步骤一：创建PAI DSW实例**，不同之处在于：

1.  你需要给新的实例命名，比如acp\_gpu，你需要确保这个名字不与运行2.7之前课程的实例重名；
    
2.  **资源规格**选择免费试用页签中的**ecs.gn7i-c8g1.2xlarge**，不用担心，你领取到的免费额度足够支持你完成2.7与2.8的课程；
    
3.  **镜像**选择**modelscope:1.21.0-pytorch2.4.0-gpu-py310-cu124-ubuntu22.04**（需要将“镜像配置”->”芯片类型“切换为GPU）。
    
4.  **当实例状态为运行中时，重复**[**计算环境准备**](https://edu.aliyun.com/course/3130200/lesson/343310285)**中的步骤二，获取大模型ACP课程的代码。**
    
5.  当完成以上步骤后，请在你新创建的GPU实例中再重新打开本章节，继续学习以下内容。
    

> 如果你在本地没有GPU环境，不建议你在本地学习本课程，代码的运行可能会失败。

6.  安装以下依赖：
    


In [ ]:
# 你需要安装以下依赖
!pip install accelerate==1.0.1 rouge-score==0.1.2 nltk==3.9.1 ms-swift[llm]==2.4.2.post2 evalscope==0.5.5rc1

7.  把模型下载到本地
    


In [3]:
!mkdir ./model 
!modelscope download --model qwen/Qwen2.5-1.5B-Instruct --local_dir './model'

打开model文件夹，就可以看到Qwen2.5-1.5B-Instruct的模型文件了。

<img src="https://alidocs.oss-cn-zhangjiakou.aliyuncs.com/res/ZWGl0wWZx7kDO34Y/img/53b3cb42-b307-4ec2-85ec-54ab5580e096.webp" width="300">

### 2.2 微调数据准备

这里已经提前准备好了一份线上教育公司相关数据库问题查询的数据集，其中大概500+作为训练集，100+作为测试集。在`resources/2_4/data.zip`文件，你需要运行以下命令进行解压：


In [ ]:
!unzip -u resources/2_4/data.zip -d resources/2_4

解压后的data文件夹在resources/2\_4中，打开data文件夹可以看到

<img src="https://alidocs.oss-cn-zhangjiakou.aliyuncs.com/res/ZWGl0wWZx7kDO34Y/img/a0c760ab-6ba2-475a-9b0e-f2ac16514824.png" width="300">

其中train.jsonl为训练集文件，test.jsonl为测试集文件。

这里介绍在模型训练时的数据分类：

*   **训练集**：用于在**训练阶段**进行模型参数的训练。
    
*   **验证集**：用于在**训练阶段**评估模型训练的效果，从而调整模型的超参数（如学习率、退火策略等）和监控过拟合情况。
    
*   **测试集**：用于在**训练阶段完成后**评估模型训练的效果。本课程提供的实验数据中，测试集涉及的数据库均没有在训练、验证数据集中出现过。
    

> 训练集文件会被按照4:1的比例拆分为训练集与验证集，测试集文件则对应测试集。


### 2.3 微调任务目标


#### 2.3.1 尝试本地模型

在标准模型上运行一次NL2SQL任务。

In [5]:
from swift.llm import (
    get_model_tokenizer, get_template, inference, ModelType,
    get_default_template_type, inference_stream
)
import torch

#你可以根据你的需要修改query（模型输入）
query = "  #背景#\n\
    数据库信息:{'column_names': [[-1, '*', 'text'], [0, 'region id', 'number'], [0, 'region name', 'text'], [1, 'country id', 'text'], [1, 'country name', 'text'], [1, 'region id', 'number'], [2, 'department id', 'number'], [2, 'department name', 'text'], [2, 'manager id', 'number'], [2, 'location id', 'number'], [3, 'job id', 'text'], [3, 'job title', 'text'], [3, 'min salary', 'number'], [3, 'max salary', 'number'], [4, 'employee id', 'number'], [4, 'first name', 'text'], [4, 'last name', 'text'], [4, 'email', 'text'], [4, 'phone number', 'text'], [4, 'hire date', 'time'], [4, 'job id', 'text'], [4, 'salary', 'number'], [4, 'commission pct', 'number'], [4, 'manager id', 'number'], [4, 'department id', 'number'], [5, 'employee id', 'number'], [5, 'start date', 'time'], [5, 'end date', 'time'], [5, 'job id', 'text'], [5, 'department id', 'number'], [6, 'location id', 'number'], [6, 'street address', 'text'], [6, 'postal code', 'text'], [6, 'city', 'text'], [6, 'state province', 'text'], [6, 'country id', 'text']], 'foreign_keys': [[5, 1], [20, 10], [24, 6], [28, 10], [29, 6], [25, 14], [35, 3]], 'primary_keys': [1, 3, 6, 10, 14, 25, 30], 'table_names': ['regions', 'countries', 'departments', 'jobs', 'employees', 'job history', 'locations']}\
    \n  #受众#\n\
    Mysql数据库\
    \n  #输出#\n\
    只输出SQL查询语句\
    \n  #目的#\n\
    将问题\"What is all the information about employees who have never had a job in the past?\"转化为SQL查询语句'"

# 获得模型模板
model_type = ModelType.qwen2_5_1_5b_instruct
model_id_or_path = "./model"
template_type = get_default_template_type(model_type)
print(f'template_type: {template_type}')  # template_type: qwen2_5
# 初始化模型
kwargs = {}
model, tokenizer = get_model_tokenizer(model_type, torch.float32, model_id_or_path=model_id_or_path, model_kwargs={'device_map': 'cpu'}, **kwargs)
model.generation_config.max_new_tokens = 128
template = get_template(template_type, tokenizer)
# 执行推理
response, history = inference(model, template, query)
# 格式化输出
print('输入：---------------------------------------------------------------------------------------------')
print(query)
print('期望输出：-----------------------------------------------------------------------------------------')
print('''  SELECT * FROM employees WHERE employee_id NOT IN (SELECT employee_id FROM job_history)''')
print('输出：---------------------------------------------------------------------------------------------')
print(response)
print('END：---------------------------------------------------------------------------------------------')
# 缓存清理
del model,template,tokenizer;import gc;_=gc.collect()

[INFO:swift] Successfully registered `/usr/local/lib/python3.10/site-packages/swift/llm/data/dataset_info.json`
[INFO:swift] Loading the model using model_dir: ./model
[INFO:swift] model_kwargs: {'device_map': 'cpu'}


template_type: qwen2_5


[INFO:swift] model.max_model_len: 32768
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


输入：---------------------------------------------------------------------------------------------
  #背景#
    数据库信息:{'column_names': [[-1, '*', 'text'], [0, 'region id', 'number'], [0, 'region name', 'text'], [1, 'country id', 'text'], [1, 'country name', 'text'], [1, 'region id', 'number'], [2, 'department id', 'number'], [2, 'department name', 'text'], [2, 'manager id', 'number'], [2, 'location id', 'number'], [3, 'job id', 'text'], [3, 'job title', 'text'], [3, 'min salary', 'number'], [3, 'max salary', 'number'], [4, 'employee id', 'number'], [4, 'first name', 'text'], [4, 'last name', 'text'], [4, 'email', 'text'], [4, 'phone number', 'text'], [4, 'hire date', 'time'], [4, 'job id', 'text'], [4, 'salary', 'number'], [4, 'commission pct', 'number'], [4, 'manager id', 'number'], [4, 'department id', 'number'], [5, 'employee id', 'number'], [5, 'start date', 'time'], [5, 'end date', 'time'], [5, 'job id', 'text'], [5, 'department id', 'number'], [6, 'location id', 'number'], [6, 'stree

#### 2.3.2 评测指标

不同类型的任务评测指标有显著差异，比如：

1.  **分类任务**：
    
    *   准确率（Accuracy）：正确预测的比例。
        
    *   精确度（Precision）、召回率（Recall）与F1分数（F1 Score）：用于衡量二分类或多分类问题中正类别的识别效果。
        
2.  **文本生成任务**：
    

    *   BLEU (Bilingual Evaluation Understudy) 分数：主要用于机器翻译等自然语言处理任务中，通过比较候选翻译与一个或多个参考翻译之间的n-gram重叠来计算得分。
        
    *   ROUGE (Recall-Oriented Understudy for Gisting Evaluation) 分数：常用于自动摘要评价，它基于n-gram召回率、精确率以及F-measure。
        
    *   Perplexity (困惑度)：用来衡量概率分布模型预测样本的不确定程度；越低越好。
    

3.  **图像识别/目标检测**：
    

    *   Intersection over Union (IoU)：两个边界框相交部分面积与并集面积之比。
        
    *   mAP (mean Average Precision)：平均精度均值，广泛应用于物体检测任务中。
    

NL2SQL任务的目的是输出一条可正确执行的SQL语句， 所以可以用“SQL语句是否输出正确”和“SQL语句执行结果是否正确”两个标准来衡量。这里我们为了实验方便，只使用“**SQL语句是否输出正确**”来评测任务。

这里再简单介绍一下ROUGE Scores (Recall-Oriented Understudy for Gisting Evaluation)：

`rouge-n-(r/p/f)`: 这是基于词级别的n-gram匹配度量，其中`n=1`，意味着考虑单个词汇的重合情况；`n=2`，意味着考虑两个词汇组成的短语的重合情况；`n=L(long)`，意味着考虑一个模型输出与测试用例中最长公共子序列的重合情况。

*   `rouge-n-r`：召回率（Recall），表示模型输出正确预测出测试用例的比例。
    
*   `rouge-n-p`：精确率（Precision），表示模型输出有多大部分也出现在测试用例中。
    
*   `rouge-n-f`：F1分数，是召回率和精确率的调和平均值，综合考量了两者的表现。
    

> 相关文章：ROUGE: A Package for Automatic Evaluation of Summaries (Chin-Yew Lin, 2004)

`Rouge-L`分数可以评估输出的SQL语句有多少部分是有用的，因此它能大致反映出模型的能力。后续我们将以`Rouge-L`作为评估指标。


#### 2.3.3 评测本地模型

这里使用ms-swift对任务进行评测。使用的评测模式为：general\_qa。该评测模式默认会计算`Rouge-1/2/L`分数。更多信息请参考：[问答题格式数据集构建方法](https://evalscope.readthedocs.io/zh-cn/latest/best_practice/swift_integration.html#general-qa)。

按照ms-swift框架要求，测试集文件每条数据遵循以下格式：


In [ ]:
{
    "history": ["表示对话历史"],
    "query": "融合了背景信息与输入问题",
    "response":  "query对应的真实答案"
}

> 本微调教程没有涉及多轮对话的数据，因此history字段为`[]`。

评测命令：


In [ ]:
!mkdir -p eval_outputs
%env LOG_LEVEL=WARNING
!swift eval \
--model_id_or_path './model' \
--model_type 'qwen2_5-1_5b-instruct' \
--eval_dataset no \
--custom_eval_config 'resources/2_4/data/config_eval.json' \
--max_length -1 \
--system '' \
--infer_backend 'pt' \
--name 'pre_train_evaluation' \
--eval_output_dir './eval_outputs'


完整参数列表请参考：[eval 命令行参数](https://swift.readthedocs.io/zh-cn/latest/Instruction/%E5%91%BD%E4%BB%A4%E8%A1%8C%E5%8F%82%E6%95%B0.html#id16)

我们关心的两个模型`Rouge-L`分数如下：（每次评测结果会有一些细微差异，这是由于大模型生成的随意性导致的）

|   |  qwen-plus  |  qwen2\_5-1\_5b-instruct  |
| --- | --- | --- |
|  rouge-l-r  |  0.5995804511766886  |   0.4730549779987757  |
|  rouge-l-p  |  0.5671844066005733  |  0.3572425303637290  |
|  rouge-l-f  |  0.5755829728881274  |  0.3907812431837991  |

我们的目标便是让微调后的`qwen2.5-1.5b-instruct`模型在NL2SQL任务上的`Rouge-L`分数相当或超过`qwen-plus`。

### 2.4 微调模型

在完成了模型的评测后，就可以开始对模型进行微调了。swift提供了操作简单的微调工具，你只需要传入训练集、模型路径等参数。同时swift框架在微调时默认使用LoRA方法，在命令中不需要额外声明。

微调命令：

In [ ]:
%env CUDA_VISIBLE_DEVICES=0
%env LOG_LEVEL=INFO
!swift sft \
--dataset 'resources/2_4/data/train.jsonl' \
--learning_rate '1e-4' \
--eval_steps '10' \
--batch_size '4' \
--model_type 'qwen2_5-1_5b-instruct' \
--max_length 2048 \
--model_id_or_path './model' \
--num_train_epochs 3

微调的完整参数列表请参考：[sft 命令行参数](https://swift.readthedocs.io/zh-cn/latest/Instruction/%E5%91%BD%E4%BB%A4%E8%A1%8C%E5%8F%82%E6%95%B0.html#id8) 

在微调时，并非所有训练样本都会直接参与到模型参数的训练中，swift框架会以4:1的比例将训练样本划分为训练集与验证集。其中训练集会直接参与到模型参数的训练；验证集虽然不直接参与训练过程，但在模型训练时提供了实时评测的效果，通过设置`--eval_steps`参数指定每训练多少steps，进行评测与保存当前模型的checkpoint（文件夹内包含“微调参数矩阵”、日志等），这样swift框架可以保存在验证集上获得最佳评测效果的checkpoint，作为最佳模型（即日志中打印的best\_model\_checkpoint）。并且在训练结束时，并不会把“微调参数矩阵”与“预训练权重矩阵”合并。

<table><tr><td bgcolor=white>
<img src="https://alidocs.oss-cn-zhangjiakou.aliyuncs.com/res/oJGq765B3YDrnAKe/img/b2e2caf3-e8da-4ac8-b57d-abbb05f932c4.png" width="800"></tr></table>

完整的loss和eval\_loss信息保存在文件`output/qwen2_5-1_5b-instruct/v2-202xxxxx-xxxx/logging.jsonl`中。

*   **如果你观察到loss在减少，eval\_loss也在减少**，说明模型需要继续训练。你可以增加`--num_train_epochs`的值来增加模型的训练深度。  
    _可以将当前的模型类比为一个处在上升期的运动员，通过不断的练习即可稳步提升比赛成绩。_
    
*   **如果你观察到loss在减少，eval\_loss却在增加，**说明模型已经处于**过拟合**状态（过度拟合了训练数据中的潜在模式）。在LoRA微调中，可以通过增加`--lora_dropout`的值，抑制模型的**过拟合**倾向。  
    _可以将当前的模型类比为一个只会背题库的学生，无法掌握题目背后的机理。_
    
*   **如果你观察到loss在增加，eval\_loss也增加，**（不常见）说明模型训练失败，需要停止训练并排查失败原因。常见的原因有：
    
    *   学习率设置过大，导致模型参数剧烈震荡。
        
    *   训练数据有问题，例如模型输入和预期输出不存在任何逻辑关系。
        
    *   抑制模型过拟合的参数设置过大，导致模型无法学习新知识。
        

你可以在output/qwen2\_5-1\_5b-instruct/v2-202xxxxx-xxxxxx/images文件夹查看训练过程中的loss、eval\_loss等指标的变化情况。


### 2.5 评测微调后的模型


#### 2.5.1 尝试微调后的模型

你可以运行以下代码，测试微调效果。  
运行以下代码前请将`ckpt_dir`修改为在验证集表现最佳的微调模型路径。微调后一般会保存两个checkpoint文件，分别是best\_model\_checkpoint（在验证集表现最佳的微调模型）与last\_model\_checkpoint（最后一次保存的checkpoint）。


In [10]:
from swift.llm import (
    get_model_tokenizer, get_template, inference, ModelType, get_default_template_type,
)
from swift.tuners import Swift
import torch

# 请在运行前修改ckpt_dir到正确的位置
ckpt_dir = 'output/qwen2_5-1_5b-instruct/vx-xxx/checkpoint-xxx<请修改为lora微调后checkpoint位置>'
# 你可以根据你的需要修改query（模型输入）
query = "  #背景#\n\
    数据库信息:{'column_names': [[-1, '*', 'text'], [0, 'region id', 'number'], [0, 'region name', 'text'], [1, 'country id', 'text'], [1, 'country name', 'text'], [1, 'region id', 'number'], [2, 'department id', 'number'], [2, 'department name', 'text'], [2, 'manager id', 'number'], [2, 'location id', 'number'], [3, 'job id', 'text'], [3, 'job title', 'text'], [3, 'min salary', 'number'], [3, 'max salary', 'number'], [4, 'employee id', 'number'], [4, 'first name', 'text'], [4, 'last name', 'text'], [4, 'email', 'text'], [4, 'phone number', 'text'], [4, 'hire date', 'time'], [4, 'job id', 'text'], [4, 'salary', 'number'], [4, 'commission pct', 'number'], [4, 'manager id', 'number'], [4, 'department id', 'number'], [5, 'employee id', 'number'], [5, 'start date', 'time'], [5, 'end date', 'time'], [5, 'job id', 'text'], [5, 'department id', 'number'], [6, 'location id', 'number'], [6, 'street address', 'text'], [6, 'postal code', 'text'], [6, 'city', 'text'], [6, 'state province', 'text'], [6, 'country id', 'text']], 'foreign_keys': [[5, 1], [20, 10], [24, 6], [28, 10], [29, 6], [25, 14], [35, 3]], 'primary_keys': [1, 3, 6, 10, 14, 25, 30], 'table_names': ['regions', 'countries', 'departments', 'jobs', 'employees', 'job history', 'locations']}\
    \n  #受众#\n\
    Mysql数据库\
    \n  #输出#\n\
    只输出SQL查询语句\
    \n  #目的#\n\
    将问题\"What is all the information about employees who have never had a job in the past?\"转化为SQL查询语句'"

# 获得模型模板
model_type = ModelType.qwen2_5_1_5b_instruct
template_type = get_default_template_type(model_type)
model_id_or_path = "./model"
# 初始化模型
model, tokenizer = get_model_tokenizer(model_type, torch.float32, model_id_or_path=model_id_or_path, model_kwargs={'device_map': 'cpu'})
model.generation_config.max_new_tokens = 128
model = Swift.from_pretrained(model, ckpt_dir, inference_mode=True)
import gc;_=gc.collect();
template = get_template(template_type, tokenizer)
# 执行推理
response, history = inference(model, template, query)
# 格式化输出
print('输入：---------------------------------------------------------------------------------------------')
print(query)
print('期望输出：-----------------------------------------------------------------------------------------')
print('''  SELECT * FROM employees WHERE employee_id NOT IN (SELECT employee_id FROM job_history)''')
print('输出：---------------------------------------------------------------------------------------------')
print(  response)
print('END：---------------------------------------------------------------------------------------------')
# 缓存清理
del model,template,tokenizer;import gc;_=gc.collect();

[INFO:swift] Loading the model using model_dir: ./model
[INFO:swift] model_kwargs: {'device_map': 'cpu'}
[INFO:swift] model.max_model_len: 32768


输入：---------------------------------------------------------------------------------------------
  #背景#
    数据库信息:{'column_names': [[-1, '*', 'text'], [0, 'region id', 'number'], [0, 'region name', 'text'], [1, 'country id', 'text'], [1, 'country name', 'text'], [1, 'region id', 'number'], [2, 'department id', 'number'], [2, 'department name', 'text'], [2, 'manager id', 'number'], [2, 'location id', 'number'], [3, 'job id', 'text'], [3, 'job title', 'text'], [3, 'min salary', 'number'], [3, 'max salary', 'number'], [4, 'employee id', 'number'], [4, 'first name', 'text'], [4, 'last name', 'text'], [4, 'email', 'text'], [4, 'phone number', 'text'], [4, 'hire date', 'time'], [4, 'job id', 'text'], [4, 'salary', 'number'], [4, 'commission pct', 'number'], [4, 'manager id', 'number'], [4, 'department id', 'number'], [5, 'employee id', 'number'], [5, 'start date', 'time'], [5, 'end date', 'time'], [5, 'job id', 'text'], [5, 'department id', 'number'], [6, 'location id', 'number'], [6, 'stree

#### 2.5.2 评测微调后的模型

（运行以下代码前请修改ckpt\_dir为在验证集表现最佳的微调模型路径）

In [ ]:
!mkdir -p eval_outputs
%env LOG_LEVEL=WARNING
!swift eval \
--ckpt_dir 'qwen2-7b-instruct/vx-xxx/checkpoint-xxx<请修改为lora微调后checkpoint位置>' \
--eval_dataset no \
--custom_eval_config 'resources/2_4/data/config_eval.json' \
--max_length -1 \
--system '' \
--infer_backend 'pt' \
--name 'pre_train_evaluation' \
--eval_output_dir './eval_outputs'

完整参数列表请参考：[eval 命令行参数](https://swift.readthedocs.io/zh-cn/latest/Instruction/%E5%91%BD%E4%BB%A4%E8%A1%8C%E5%8F%82%E6%95%B0.html#id16)

而我们把微调后模型的分数与之前的参考分数进行对比：

|   |  qwen-plus  |  qwen2\_5-1\_5b-instruct  |  qwen2\_5-1\_5b-instruct(微调后)  |
| --- | --- | --- | --- |
|  rouge-l-r  |  0.5995804511766886  |   0.4730549779987757  |  0.6540312152096097  |
|  rouge-l-p  |  0.5671844066005733  |  0.3572425303637290  |  0.6203511702379731  |
|  rouge-l-f  |  0.5755829728881274  |  0.3907812431837991  |  0.6202922260470989  |

可以看到，`Rouge-L`分数都有了非常大的提升，几乎翻倍，性能甚至已经超过了`qwen-plus`。说明经过微调后的模型NL2SQL的能力得到了大幅度的加强。


### 2.6 参数矩阵融合

模型微调训练完成后，有两种方式可以使用微调后的模型：

1.  在调用时动态加载微调模型。  
    微调后获得的低秩参数矩阵只占20MB的存储空间，这个大小非常便于做增量发布和传播，这也是工程上常用的方法。需要注意的是，用哪个基础模型微调，在加载时就需要指定使用哪个基础模型。  
    我们在前一小节中，已经通过指定`ckpt_dir`尝试了这种方法。
    
2.  将基础模型与微调得到的低秩参数融合，获得一个完整的、更新了参数的模型，再调用融合了的模型。
    

这里我们介绍第二种方法：融合“微调参数矩阵”与”基础模型参数矩阵“，将微调后的模型参数存储成一个完整的参数矩阵。

通过`swift export`方法，传入微调模型的路径（建议传入`best_model_checkpoint`），即可得到融合后的模型。


In [ ]:
%env LOG_LEVEL=INFO
!swift export \
--ckpt_dir 'output/qwen2_5-1_5b-instruct/vx-xxx/checkpoint-xxx<请修改为lora微调后checkpoint位置>' \
--merge_lora true

日志展示了融合后模型的路径。融合后的完整参数矩阵默认存储在`checkpoint`目录下。（PAI实验环境完整的模型参数在：`output/qwen2_5-1_5b-instruct/vX-XXX/checkpoint-XX-merged`）。
    

## ✅ 本节小结

在本节课程中，我们学习了以下内容：

*   了解模型小型化的应用场景、常见方式及其限制。
    
*   学习了模型高效微调的几种常用方法，并重点对LoRA高效微调的过程和优点进行了讨论。
    
*   最后通过一个LoRA微调实验，我们简单地进行了一次完整的微调。
    
*   学习了基本的模型评估标准，并在微调前后进行了模型评测和结果对比。

由于本课程第2\_9章节"部署模型到生产环境"的内容涉及到模型部署，也需要GPU示例，因此你可以继续留在这个GPU实例中学习2\_9的课程内容。在2\_9的教程中，你将了解如何将本地模型（是否经过微调都可以）部署到阿里云ECS或PAI上。如果你要部署微调模型，需要部署参数矩阵融合后生成的完整参数矩阵（后缀带merged），格式为：output/qwen2\_5-1\_5b-instruct/v2-202xxxxx-xxxxxx/checkpoint-xxx-merged。


## 扩展阅读


### 开展微调任务前必须要关注的

虽然在本教程中你可以使用准备好的NL2SQL数据集，免费体验GPU算力资源进行微调，但在实际生产中，微调往往并不简单。高昂的算力成本与对数据规模和质量的严苛要求都是你在微调前需要考虑的因素。

在开始微调任务之前，建议你关注以下方面：

1.  明确业务需求匹配度  
    首先，你需要清楚业务的具体需求，明确微调要解决的具体问题，并评估当前模型是否已能满足大部分需求，如果是，则可能不需要微调。确定是否有明确的业务指标来衡量微调前后的效果，例如NL2SQL的查询成功率。
    
2.  尝试更大的模型  
    你可以考虑使用参数更大的模型，因为它们经过更多数据和训练轮次，在通用领域和特定领域都可能带来显著的性能提升。 即使有使用成本和延迟限制，也建议进行实验或性能验证。更大的模型在微调时可以作为“老师”，传授知识给较小的“学生”模型。
    
3.  使用提示词工程和插件调用  
    这种方法比对模型进行微调更灵活、成本更低，并且这些前期工作可以在构建微调数据集时复用。
    
4.  评估资源和技术可行性  
    微调过程需要计算资源和时间成本，包括GPU资源（训练时需要）、存储空间（存储模型时需要）和专家人力成本（构建数据集时需要）。评估项目预算和资源是否允许进行有效微调，同时确保团队具备所需的技术能力，或寻求合作伙伴提供支持（你也可以直接使用[百炼提供的0代码微调能力](https://help.aliyun.com/zh/model-studio/user-guide/using-fine-tuning-on-console)）。
    
5.  确保充足的数据  
    虽然微调相比于预训练阶段对数据要求降低，但仍需准备足够的精确标注数据集，通常需要至少1000条数据才能获得良好效果。
    
6.  关注合规与隐私  
    确保使用的数据符合法律法规要求，处理个人数据时遵循隐私保护原则。
    



### 微调数据集构建策略

一般来说，在比较复杂的场景中，微调至少需要**1000+条优质的训练集数据**。构建数据集时，请确认以下几点：

*   **数据质量**：确保数据集的准确性和相关性，避免模糊和错误内容。
    
*   **数据多样性**：覆盖任务的所有关键方面和潜在变化，包括不同场景、语境和专业术语。
    
*   **平衡性**：如果任务涉及多种类别场景，确保各类别样本均衡，防止模型偏向于某一类。
    
*   **持续迭代**：微调是一个迭代过程，根据模型在验证集上的表现反馈，不断优化和扩大数据集。
    

而如果你在进行模型微调时缺乏数据，建议你使用知识库检索来增强模型能力。

> 在很多复杂的业务场景中，可以综合采用模型调优和知识库检索相结合的技术方案。

你也可以采用以下策略扩充数据集：

1.  联系数据专家或数据团队基于已有数据，制作更多的典型场景数据。
    
2.  让能力更强的大模型模拟生成特定业务/场景的相关内容，辅助你生成更多可用于微调的数据。
    
3.  通过应用场景收集、网络爬虫、社交媒体和在线论坛、公开数据集、合作伙伴与行业资源、用户贡献等方式获取更多数据。
    


## ✉️ 评价反馈
感谢你学习阿里云大模型ACP认证课程，如果你觉得课程有哪里写得好、哪里写得不好，期待你[通过问卷提交评价和反馈](https://survey.aliyun.com/apps/zhiliao/Mo5O9vuie)。

你的批评和鼓励都是我们前进的动力。

## 🔥 课后小测验

【单选题】2.4.1. 以下关于 LoRA 的描述，哪一项是错误的？（ ）

A. LoRA 可以有效降低微调大型语言模型的成本。

B. LoRA 会修改被微调模型的原始权重。

C. LoRA 的实现相对简单，易于集成。

D. LoRA 微调的结果可以方便地回退。

答案：B

解析： LoRA 并不直接修改原始权重，而是通过添加低秩矩阵间接影响模型行为。 这使得回退操作变得简单，只需移除添加的低秩矩阵即可。

【多选题】2.4.2. 你正在使用 Swift 微调一个 Qwen 模型，发现模型在验证集上的 loss 出现了明显的上升趋势，以下哪些操作可以帮助你缓解或解决这个问题？( )

A. 增大 --learning\_rate

B. 减小 --learning\_rate

C. 增大 --lora\_dropout

D. 减小 --lora\_dropout

E. 增大 --num\_train\_epochs

F. 减小 --num\_train\_epochs

答案: B, C, F

解释:

*   learning\_rate: 学习率过大会导致模型训练速度快，但可能在最优解附近震荡，甚至无法收敛，导致loss波动，看起来像是过拟合。当然这与真正的过拟合不同。
    
*   lora\_dropout: 增大 dropout 率可以增强模型的泛化能力，抑制过拟合。
    
*   num\_train\_epochs: 过拟合也可能是训练次数过多导致的，减少训练次数可以避免模型过度学习训练数据。